- ดึงชื่ออนิเมนะปี2025
- Cell 0: import ไลบรารีที่ใช้ (json, pandas, MongoClient)


In [42]:
import json
import pandas as pd
from pymongo import MongoClient
import requests
import time

In [ ]:

# --- การตั้งค่าเบื้องต้น ---
YEAR = 2025
SEASONS = ["winter", "spring", "summer", "fall"]
BASE_URL = "https://api.jikan.moe/v4/seasons"
HEADERS = {"User-Agent": "anime-fetcher/1.0"}

# เตรียมโครงสร้างข้อมูลสำหรับเก็บผลลัพธ์
anime_2025 = {
    "year": YEAR,
    "seasons": {}
}

# --- เริ่มการทำงานหลัก ---
for season in SEASONS:
    all_anime = []
    page = 1
    
    while True:
        url = f"{BASE_URL}/{YEAR}/{season}?page={page}"
        print(f"Fetching {season} {YEAR} | page {page}")

        try:
            res = requests.get(url, headers=HEADERS, timeout=10)
            
            if res.status_code == 429: # กรณีโดน Rate Limit (ส่งคำขอเร็วเกินไป)
                print("Rate limit hit, waiting 5 seconds...")
                time.sleep(5)
                continue
                
            if res.status_code != 200:
                print(f"HTTP {res.status_code} Error: {res.text[:100]}")
                break

            data = res.json()
            
            if "data" not in data:
                break

            all_anime.extend(data["data"])

            # ตรวจสอบว่ามีหน้าถัดไปหรือไม่
            if not data.get("pagination", {}).get("has_next_page", False):
                break

            page += 1
            time.sleep(1.5) # รอระหว่างหน้า (API Limit: 3 requests per second)

        except requests.exceptions.RequestException as e:
            print(f"Connection error: {e}")
            break

    # เก็บข้อมูลที่ดึงได้ลงใน Dictionary หลัก
    anime_2025["seasons"][season] = all_anime
    print(f'add {season}')
    print(f"Completed {season}: {len(all_anime)} items found.\n")
    
    # พักสั้นๆ ก่อนเริ่ม Season ถัดไป
    time.sleep(2)


Fetching winter 2025 | page 1
Fetching winter 2025 | page 2
Fetching winter 2025 | page 3
Fetching winter 2025 | page 4
Fetching winter 2025 | page 5
Fetching winter 2025 | page 6
Fetching winter 2025 | page 7
Fetching winter 2025 | page 8
Fetching winter 2025 | page 9
Fetching winter 2025 | page 10
Fetching winter 2025 | page 11
add {seseason}
Completed winter: 253 items found.

Fetching spring 2025 | page 1
Fetching spring 2025 | page 2
Fetching spring 2025 | page 3
Fetching spring 2025 | page 4
Fetching spring 2025 | page 5
Fetching spring 2025 | page 6
Fetching spring 2025 | page 7
Fetching spring 2025 | page 8
Fetching spring 2025 | page 9
Fetching spring 2025 | page 10
Fetching spring 2025 | page 11
add {seseason}
Completed spring: 265 items found.

Fetching summer 2025 | page 1
Fetching summer 2025 | page 2
Fetching summer 2025 | page 3
Fetching summer 2025 | page 4
Fetching summer 2025 | page 5
Fetching summer 2025 | page 6
Fetching summer 2025 | page 7
Fetching summer 2025 | p

In [ ]:
print(anime_2025['seasons'].get('fall'))

4
[{'mal_id': 52807, 'url': 'https://myanimelist.net/anime/52807/One_Punch_Man_3', 'images': {'jpg': {'image_url': 'https://cdn.myanimelist.net/images/anime/1168/148347.jpg', 'small_image_url': 'https://cdn.myanimelist.net/images/anime/1168/148347t.jpg', 'large_image_url': 'https://cdn.myanimelist.net/images/anime/1168/148347l.jpg'}, 'webp': {'image_url': 'https://cdn.myanimelist.net/images/anime/1168/148347.webp', 'small_image_url': 'https://cdn.myanimelist.net/images/anime/1168/148347t.webp', 'large_image_url': 'https://cdn.myanimelist.net/images/anime/1168/148347l.webp'}}, 'trailer': {'youtube_id': None, 'url': None, 'embed_url': 'https://www.youtube-nocookie.com/embed/2GU7Ye78h6E?enablejsapi=1&wmode=opaque&autoplay=1', 'images': {'image_url': None, 'small_image_url': None, 'medium_image_url': None, 'large_image_url': None, 'maximum_image_url': None}}, 'approved': True, 'titles': [{'type': 'Default', 'title': 'One Punch Man 3'}, {'type': 'Synonym', 'title': 'One Punch Man 3rd Season


- Cell 3: สร้างรายการ rows โดยวนรอบแต่ละ season ใน data['seasons'] และสำหรับแต่ละ anime สร้าง dict ที่เก็บฟิลด์สำคัญ (mal_id, url, season, year, trailer, title, title_english, type, source, rating, score, genres) แล้ว append ลง rows


In [64]:
rows = []

for season, anime_list in anime_2025['seasons'].items():
    for anime in anime_list:
        rows.append({
            "mal_id": anime["mal_id"],
            "url": anime["url"],
            "season": season,
            "year": anime_2025["year"],
            "trailer":anime['trailer'],
            "title": anime['title'],
            "title_english":anime["title_english"],
            "type" : anime['type'],
            "source":anime['source'],
            "rating":anime['rating'],
            "score" : anime['score'],
            "genres":anime['genres']
        })



- Cell 4: ใช้ displayเพื่อดูข้อมูลที่สร้างขึ้น

In [71]:
print(len(rows))
display(rows)

1059


[{'mal_id': 58567,
  'url': 'https://myanimelist.net/anime/58567/Ore_dake_Level_Up_na_Ken_Season_2__Arise_from_the_Shadow',
  'season': 'winter',
  'year': 2025,
  'trailer': {'youtube_id': None,
   'url': None,
   'embed_url': 'https://www.youtube-nocookie.com/embed/GDMXGzjJzS4?enablejsapi=1&wmode=opaque&autoplay=1',
   'images': {'image_url': None,
    'small_image_url': None,
    'medium_image_url': None,
    'large_image_url': None,
    'maximum_image_url': None}},
  'title': 'Ore dake Level Up na Ken Season 2: Arise from the Shadow',
  'title_english': 'Solo Leveling Season 2: Arise from the Shadow',
  'type': 'TV',
  'source': 'Web manga',
  'rating': 'R - 17+ (violence & profanity)',
  'score': 8.58,
  'genres': [{'mal_id': 1,
    'type': 'anime',
    'name': 'Action',
    'url': 'https://myanimelist.net/anime/genre/1/Action'},
   {'mal_id': 2,
    'type': 'anime',
    'name': 'Adventure',
    'url': 'https://myanimelist.net/anime/genre/2/Adventure'},
   {'mal_id': 10,
    'type



- Cell 5: เชื่อมต่อ MongoDB เพื่อบันทึกข้อมูล



In [ ]:
try:
    client = MongoClient("mongodb://localhost:27017/")
    db = client["Dataen"]
    collection = db["AnimeList"]
    
    if rows:
        result = collection.insert_many(rows)
        print(f"Success! Inserted {len(result.inserted_ids)} anime records.")
    else:
        print("No data found to insert.")
        
except Exception as e:
    print(f" An error occurred: {e}")
finally:
    client.close()